In [3]:
import pandas as pd

C:\Users\aq75iwit\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.2' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [4]:
PATH='D:/01_Diss_Data/'

sample = pd.read_pickle(PATH+'data_transcripts_28_09_22.pkl')

In [1]:
df=sample

NameError: name 'sample' is not defined

In [46]:
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# Download the VADER lexicon if it is not already downloaded
nltk.download('vader_lexicon')

# Load the Loughran and McDonald financial sentiment dictionary
lm_dict = pd.read_csv('D:/01_Diss_Data/sentiments/Loughran-McDonald_MasterDictionary_1993-2021.csv')

# Create a function to calculate the sentiment scores for each earnings call transcript
def get_sentiment(text):
    # Use the VADER sentiment analysis tool to calculate sentiment scores
    sid = SentimentIntensityAnalyzer()
    sentiment = sid.polarity_scores(text)
    
    # Use the Loughran and McDonald financial sentiment dictionary to adjust the sentiment scores
    lm_score_pos = sum([lm_dict.loc[lm_dict['Word'] == word.lower(), 'Positive'] 
                        for word in text.split() if word.lower() in lm_dict.index])
    lm_score_neg = sum([lm_dict.loc[lm_dict['Word'] == word.lower(), 'Negative'] 
                        for word in text.split() if word.lower() in lm_dict.index])
    adjusted_positivity = sentiment['pos'] + lm_score_pos
    adjusted_negativity = sentiment['neg'] + lm_score_neg
    
    return adjusted_positivity, adjusted_negativity

# Load the dataframe with the earnings call transcripts
#df = pd.read_csv('earnings_calls.csv')

# Apply the get_sentiment function to the 'general' column and create new columns with the adjusted positivity and negativity scores
result = df['general'].apply(lambda x: pd.Series(get_sentiment(x)))
new_columns = ['adjusted_positivity', 'adjusted_negativity']
for i, col in enumerate(new_columns):
    df[col] = result[i]
    df[col] = df[col].astype(float)
# Apply the get_sentiment function to the 'qanda_manager_text' and 'qanda_analyst_text' columns and create new columns with the adjusted positivity and negativity scores
df[['qanda_manager_adjusted_positivity', 'qanda_manager_adjusted_negativity']] = df['qanda_manager_text'].apply(lambda x: pd.Series(get_sentiment(' '.join(x))))
df[['qanda_analyst_adjusted_positivity', 'qanda_analyst_adjusted_negativity']] = df['qanda_analyst_text'].apply(lambda x: pd.Series(get_sentiment(' '.join(x))))

# Create new columns for the adjusted positivity and negativity scores for the first and second halves of the 'qanda_manager_text' column
df[['qanda_manager_positivity_first_half', 'qanda_manager_negativity_first_half']] = df['qanda_manager_text'].apply(lambda x: pd.Series(get_sentiment(' '.join(x[:len(x)//2]))))
df[['qanda_manager_positivity_second_half', 'qanda_manager_negativity_second_half']] = df['qanda_manager_text'].apply(lambda x: pd.Series(get_sentiment(' '.join(x[len(x)//2:]))))

# Create new columns for the adjusted positivity and negativity scores for the first and second halves of the 'qanda_analyst_text' column
df[['qanda_analyst_positivity_first_half', 'qanda_analyst_negativity_first_half']] = df['qanda_analyst_text'].apply(lambda x: pd.Series(get_sentiment(' '.join(x[:len(x)//2]))))
df[['qanda_analyst_positivity_second_half', 'qanda_analyst_negativity_second_half']] = df['qanda_analyst_text'].apply(lambda x: pd.Series(get_sentiment(' '.join(x[len(x)//2:]))))

# View the dataframe with the new columns
df.head()

# View the dataframe with the new 'adjusted_positivity' and 'adjusted_negativity' columns




[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\aq75iwit\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
<ipython-input-46-830072cd74c8>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = result[i]
<ipython-input-46-830072cd74c8>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(float)
<ipython-input-46-830072cd74c8>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

,filename,company_name,ticker_symbol,sector,year,quarter,perspective,release,release_time,iso_date,...,qanda_analyst_adjusted_positivity,qanda_analyst_adjusted_negativity,qanda_manager_positivity_first_half,qanda_manager_negativity_first_half,qanda_manager_positivity_second_half,qanda_manager_negativity_second_half,qanda_analyst_positivity_first_half,qanda_analyst_negativity_first_half,qanda_analyst_positivity_second_half,qanda_analyst_negativity_second_half
0,1000041-nupathes-ceo-discusses-q2-2013-results...,NuPathe,PATH,healthcare,2013,Q2,individual,"November 12, 2012",8:30 a.m.,2012-11-12T13:30:00.000,...,0.108,0.010,0.089,0.017,0.126,0.010,0.147,0.020,0.074,0.000
1,1000091-lockheed-martins-ceo-hosts-leadership-...,Lockheed Martin,LMT,industrial-goods,2012,NaN,regular,"November 12, 2012",8:30 am,2012-11-12T13:30:00.000,...,0.133,0.003,0.148,0.012,0.158,0.023,0.082,0.000,0.176,0.006
2,1000191-sky-mobi-limiteds-ceo-discusses-f2q13-...,Sky-mobi Limited,MOBI,technology,2012,Q2,regular,"November 12, 2012",08:00 am,2012-11-12T13:00:00.000,...,0.139,0.000,0.144,0.020,0.174,0.006,0.171,0.000,0.093,0.000
3,1000351-tmx-groups-ceo-discusses-q3-2012-resul...,TMX Group Inc.,TMXGF-PK,technology,2012,Q3,regular,"November 9, 2012",8:00 am,2012-11-09T13:00:00.000,...,0.088,0.005,0.164,0.021,0.120,0.030,0.095,0.009,0.072,0.000
4,100040-wells-fargo-q3-2008-earnings-call-trans...,Wells Fargo &amp; Company,WFC,financial,2008,Q3,regular,"October 15, 2008",8:30 am,2008-10-15T12:30:00.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
